### Test spatial distribution of molecular clusters:

1) to determine the spatiall distribution of molecular cell types  (a.k.a. whether they are clustered, dispersed or uniformly distributed), we compared the cell types with a CSR (complete spatial randomness) process and performed a monte carlo test of CSR (Cressie; Waller). We simulated the CSR process by randomly sampling cells in the data 1,000 times to generate a distribution of the averaged distance to nearest neighbor under CSR (ANNCSR). The number of random sampled cells was matched to that in each molecular cell type. The ANN from each molecular cell types (ANNMol) was calculated and compared to the CSR distribution to calculate the p-value. 

2) to determine whether the molecular cell types are enriched within proposed subregions, we used an approach similar to the Quadrat statistic (Cressie; Waller), instead of quadrat, the proposed anatomical parcellations are used for this analysis. One hypothesis was that the unequal distributions of molecular types within propose LHA subdomains are due to differences in cell/point densities in these subregions. To test this, we simulated the distribution by shuffling neurons' molecular identity 1000 times to compute the distribution of the χ 2 statistics for each cell type. The χ 2 statistic from the observed molecular cell types was compared to the distribution of expected χ 2 statistics under the above hypothesis to calculate the p values. 

3) to determine which subregion the given molecular cluster is enriched in, we performed the permutation test, where we shuffled the position of neurons from each molecular type 1,000 times and calculated the distribution of regional enrichment for any given molecular cell type. The observed fraction of neurons enriched in a given subregion from each molecular cell type was compared to the expected distribution from the random process to calculate the p values.

In [2]:
import os, sys
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from os.path import abspath, dirname
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('default')
from scipy import stats, spatial
import seaborn as sns
from scipy.stats import kde, pearsonr
from sklearn.utils import shuffle
#import scanpy as sc

In [3]:
lha_neuron=pd.read_csv('directory/spotcount/neuron',sep=',', index_col=0)
ex_m=pd.read_csv('/Slc17a6/molecular/type/metadata',sep=',', index_col=0)
inh_m=pd.read_csv('/Slc32a1/molecular/type/metadata',sep=',', index_col=0)
lha_neuron=lha_neuron.T
lha_neuron=lha_neuron.where(lha_neuron>=0, 0)
roi=pd.read_csv('directory/roi/metadata',sep=',', index_col=0)

cluster=pd.concat([ex_m,inh_m],axis=0)

c=['Ex-1', 'Ex-2', 'Ex-3', 'Ex-4',
   'Ex-5', 'Ex-6', 'Ex-7', 'Ex-8', 
   'Ex-9', 'Ex-10', 'Ex-11', 'Ex-12', 
   'Ex-13', 'Ex-14', 'Ex-15', 'Ex-16', 
   'Ex-17', 'Ex-18', 'Ex-19', 'Ex-20',
   'Ex-21', 'Ex-22', 'Ex-23', 'Ex-24', 
   'Ex-25','Inh-1', 'Inh-2','Inh-3', 'Inh-4', 
   'Inh-5', 'Inh-6', 'Inh-7', 'Inh-8', 'Inh-9', 
   'Inh-10', 'Inh-11', 'Inh-12', 'Inh-13', 'Inh-14',
   'Inh-15', 'Inh-16', 'Inh-17', 'Inh-18', 'Inh-19',
   'Inh-20', 'Inh-21', 'Inh-22', 'Inh-23']

###### Generate random distribution and compute ANN

In [7]:
distrib=pd.DataFrame(np.empty([len(c),1000]),index=c,columns=range(1,1001))
for n in c:  
    for i in range(1,1001):
        idx=np.random.choice(roi.index,df.loc[n,'size'].astype('int'))
        X=roi[roi.index.isin(idx)]
        dist,r=spatial.KDTree(X.to_numpy()[:,:3]).query(X.to_numpy()[:,:3], k=2)
        distrib.loc[n,i]=np.mean(dist[dist!=0])

In [12]:
matrix=pd.DataFrame(np.empty([len(c),0]),index=c)
for n in c:
    C=roi[roi.index.isin(cluster[cluster.x==n].index)].to_numpy()[:,:3]
    dist,r=spatial.KDTree(C).query(C, k=2)
    matrix.loc[n,'ANN']=np.mean(dist[dist!=0])

In [18]:
csr_test=pd.DataFrame(np.empty([len(c),0]),index=c)
csr_test.loc[j,'p_value']=-1
csr_test.loc[j,'diff']=-1
for j in c:
    d=distrib.loc[j].to_numpy()
    a=len(d[d<=matrix.loc[j,'ANN']])
#     b=1001-a
    csr_test.loc[j,'p_value']=a/1001
    csr_test.loc[j,'diff']=matrix.loc[j,'ANN']-distrib.loc[j].min()

###### χ 2 test

In [22]:
img=imread('LHA/parcellation/mask')

In [1]:
A=roi.copy()
A=A[(A.x<777)&(A.y<772)&(A.z<266)]
# roi.loc[:,'subregion']=0
lb=np.unique(img[img!=0])
df_q=pd.DataFrame(np.zeros([len(c),len(lb)]),index=c,columns=lb)
for j in c:
    C=A[A.index.isin(cluster[cluster.x==j].index)]
    for x in C.index:
        coord=np.array(np.floor(C.loc[x].to_numpy()[:3])-1)
        C.loc[x,'subregion']=img[tuple(coord)]
        roi.loc[x,'subregion']=img[tuple(coord)]
    if len(C)>0:
        for y in lb:
            df_q.loc[j,y]=len(C[C['subregion']==y])

###### Shuffle data and compare spatial distribution within LHA parcellations

In [26]:
from sklearn.utils import shuffle

a={}

for j in c:
    shuffle_s=pd.DataFrame(np.zeros([1000,len(lb)]),columns=lb)
    for ind in range (0,1000):
        roi_s=shuffle(roi.subregion.to_numpy())
        roi_shuffle=roi.copy()
        roi_shuffle['subregion']=roi_s
        X=roi_shuffle[roi_shuffle.index.isin(cluster[cluster.x==j].index)]
        if len(X)>0:
            for y in lb:
                shuffle_s.loc[ind,y]=len(X[X['subregion']==y])
        ind+=1
    a[j]=shuffle_s

In [29]:
for j in c:

    a[j]=a[j].rename(columns={1.0: "LHAd-db",3.0: "LHAdl",4.0: "LHAs-db",5.0: "ZI", 6.0: "EP",
                          7.0: "fornix",9.0: "LHA-vl",11.0:"LHAf",17.0:"LHAhcrt-db"})

    a[j]=a[j][['ZI', 'LHAd-db','LHAhcrt-db','LHAdl','LHAf','fornix', 'LHAs-db','LHA-vl','EP']]


    a[j]=a[j].drop(columns='fornix')


    a[j]=a[j].rename(columns={"LHA-vl":"LHAf-l"})

In [110]:

chi_square_shuffle=pd.DataFrame(np.zeros([len(c),1000]),index=c)
for i in c:
    for ind in range(0,1000):
        chi_square_shuffle.loc[i,ind]=stats.mstats.chisquare(a[i].loc[ind,:])[0]



In [111]:
for i in c:
    d=stats.chisquare(df_q.loc[i,:])[0]
    chi_square.loc[i,'r_pval']=len(np.where(chi_square_shuffle.loc[i,:]>d)[0])/1000


###### permutation (shuffle) test to determine which LHA subregion molecular cell types are enriched in

In [ ]:
A=roi.copy()
A=A[(A.x<777)&(A.y<772)&(A.z<266)]
roi.loc[:,'subregion']=0
lb=np.unique(img[img!=0])
df=pd.DataFrame(np.zeros([len(c),len(lb)]),index=c,columns=lb)
for j in c:
    C=A[A.index.isin(cluster[cluster.x==j].index)]
    for x in C.index:
        coord=np.array(np.floor(C.loc[x].to_numpy()[:3])-1)
        C.loc[x,'subregion']=img[tuple(coord)]
        roi.loc[x,'subregion']=img[tuple(coord)]
    if len(C)>0:
        for y in lb:
            df.loc[j,y]=len(C[C.subregion==y])/len(C)

df=df.rename(columns={1.0: "LHAd-db",3.0: "LHAdl",4.0: "LHAs-db",5.0: "ZI", 6.0: "EP",
                      7.0: "fornix",9.0: "LHAf-l",11.0:"LHAf",17.0:"LHAhcrt-db"})

df=df[['ZI', 'LHAd-db','LHAhcrt-db','LHAdl','LHAf','fornix', 'LHAs-db','LHAf-l','EP']]



In [ ]:
A=roi.copy()
A=A[(A.x<777)&(A.y<772)&(A.z<266)]
A.loc[:,'shuffle']=1
for i in c:
    m[i]=np.zeros([len(lb)+1,1])
for ind in range(1,1001):
    cluster_shuffle=cluster.copy()
    cluster_shuffle.x=shuffle(cluster.x.to_numpy())
    for i in c:
        ct=A[A.index.isin(cluster_shuffle[cluster_shuffle.x==i].index)]
        x=pd.DataFrame(data=np.zeros([len(lb)+1,1]),index=[0,1,3,4,5,6,7,9,11,17],
                       columns=['shuffle'])
        y=ct.groupby('subregion').sum()
        for j in y.index:
            x.loc[j,'shuffle']=y.loc[j,'shuffle']/len(ct)
        m[i]=np.append(m[i],x.to_numpy().reshape(10,1),axis=1)

In [ ]:
df_p=pd.DataFrame(data=np.ones(df.shape),index=df.index,columns=df.columns)

df_p.shape

ind=0
for i in c:
    
    print(i)
    for n in range(0,9):
        print(n,ind)
        df_p.iloc[ind,n]=len(np.where(m[i][n,1:]>df.iloc[ind,n])[0])/1000
    ind+=1
df_p=df_p.reindex(df_p.index[a.dendrogram_col.reordered_ind])
df_p=df_p[df_p.columns[::-1]]
df_p=df_p.drop(columns='fornix')